<a href="https://colab.research.google.com/github/limkaram/SKtelecom-TANGO-keyword-analysis/blob/master/TANGO_%EA%B5%AC%EC%B6%95_%EC%9E%91%EC%97%85_%ED%82%A4%EC%9B%8C%EB%93%9C_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google Drive 목록 Mounting(매 코딩 시작전 해주어야함)
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_excel('/content/drive/My Drive/Personal Project/TANGO 구축 작업 키워드 분석/20200101-20200731_TANGO 회선영향없음 키워드 주간 구축 작업.xlsx')
# 분석 예정 컬럼 = ['작업명']
data['작업명'].loc[4393]  # 만들어 놓은 정규식 패턴에 맞지 않는 작업명

In [ ]:
# 정규식 패턴에 부합하는 작업명 추출 함수
import re
def get_regex_matching_text(text):
    text_in_square_backet_regex = re.compile(r'\[{1}.+\]{1}')  # 정규식 패턴 : 대괄호내 어떠한 단어라도 들어가있으면 부합
    matched_text = text_in_square_backet_regex.search(text)
    if matched_text:  # 정규식 패턴에 부합하는 작업명을 지닌 경우
        return matched_text.group()
    else:  # 정규식 패턴에 부합하는 부분을 전혀 가지지 않은 작업명의 경우 None을 반환하므로, pass
        pass

In [ ]:
data[data['작업명'].apply(get_regex_matching_text).isnull() == True]  # 정규식패턴에 맞지 않는 row 확인(index = 4393)
data['정제_작업명'] = data['작업명'].apply(get_regex_matching_text)  # 정규식화 함수를 거친 새로운 columns 생성

# 대괄호 제거 후 내부 단어만 추출
def get_rid_of_square_bracket(text):
    if text:  # refined_text가 None이 아닌 경우만 대괄호 제거 후 정제
        return text.replace('][', '|').lstrip('[').rstrip(']').split('|')
    else:
        return 0
data['정제_작업명'] = data['정제_작업명'].apply(get_rid_of_square_bracket)
data['정제_작업명'].loc[4390:4400]

In [ ]:
# '정제_작업명' column에서 회선영향없음 포함하는 값은 삭제
def re_refinement(value):
    if type(value) is list:
        return [i for i in value if len(i) != 1 if '회선영향없음' not in i]
    else:
        pass

data['정제_작업명'] = data['정제_작업명'].apply(re_refinement)  # 최종 정제된 단어 리스트를 value로 가지는 column 완성

In [ ]:
# 시각화 한글 깨짐 해소
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf


In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
plt.rc('font', family='NanumBarunGothic')  # 한글 폰트 깨짐 해소

# 텍스트별 갯수 시각화
job_title_list = []

for job_title in data['정제_작업명']:  # 정제_작업명 value dtype is list
    if job_title:
        for title in job_title:
            job_title_list.append(title)

for index, value in enumerate(job_title_list):
    if len(value) > 7:
        # print(value)
        del(job_title_list[index])

plt.figure(figsize=(16,6))
plt.subplot()
sns.countplot(x=job_title_list)
plt.show()


In [ ]:
# 키워드들의 종류가 너무 많아 시각화 분석이 힘드므로, 빈도수 분석부터 시행
import collections

keyword_counter = collections.Counter(job_title_list)
keyword_counter = keyword_counter.most_common()

In [ ]:
# 빈도율 삽입
# 전체 row 갯수 : 4954
dataframe_form_dict = {'키워드': [],
                       '빈도수': [],
                       '빈도율(%)': []}
for word, freq in keyword_counter:
    freq_rate = round(freq / 4954 * 100)  # 단어 빈도율 계산
    freq_rate_str = str(freq_rate) + '%'  # 단어 빈도율 문자열로 변경 후 % 붙힘
    dataframe_form_dict['키워드'].append(word)
    dataframe_form_dict['빈도수'].append(freq)
    dataframe_form_dict['빈도율(%)'].append(freq_rate_str)
# print(dataframe_form_dict)

result_df = pd.DataFrame(dataframe_form_dict)
result_df.head()

In [ ]:
# 데이터 전처리(불필요키워드 제거)
result_df['키워드'].unique()  # 전체 키워드 확인

unnecessary_keyword = [word for word in list(result_df['키워드']) if ' ' in word] # 단어 사이에 띄워쓰기 있는 불필요 키워드 발췌
unnecessary_keyword += ['실사', '긴급', '접속', 'SKB', '중심국출입', '선번조사', '발란싱', '옥외랙', '긴급]', '위탁사업', '중심국출입요청', '군포통합국', 'OJC', 'IPD', 'PCM']  # 불필요 단어 발췌 후 추가

for keyword in unnecessary_keyword:  # 전체 발췌 불필요 키워드 제거
    result_df = result_df[result_df['키워드'] != keyword]

result_df.head(10)

In [ ]:
# 데이터 전처리(A망 키워드 제거)
result_df['키워드'].unique()
access_keyword = ['5G-PON', '링먹스', '5GPON', 'SMUX', '5G-PON2', 'CPRIMW', 'S-DDF', 'SDDF', 'DDF', 'S-MUX', '선로점검']  # A망 관련 키워드
for keyword in access_keyword:
    result_df = result_df[result_df['키워드'] != keyword]
result_df = result_df.reset_index(drop=True)
result_df.head(10)

In [ ]:
# iRMS 관련 키워드로만 구성
irms_keywords = ['정류기', '부대공사', '부대물자', 'RMS', '부대시설', '축전지', '리튬']

irms_df = pd.DataFrame(columns=result_df.columns)
# irms_df.head()
for keyword in irms_keywords:
    irms_df = irms_df.append(result_df[result_df['키워드'] == keyword])
irms_df.reset_index(drop=True)